*ToDo*

02) pred_window und obs_window ausprobieren.
03) output nochmal genau anschauen.
04) was ist maskiert anschauen - und wie genau?
05) VRAM wenig ausgelastet. Batch size mal mit 320 ausprobieren, aber auch durchziehen.
06) Task pred: scheduler einfügen.
07) Analyse each var (sparsity)
08) loss per var/quality of varis loss (is this already ablation to forecast only one var)
09) Include sepsis definition
10) 

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import joblib
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
def set_seed(seed: int) -> None:
    """
    Set the random seed for modules torch, numpy and random.

    :param seed: random seed
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed_all(seed)
set_seed(42)

## Load forecast dataset into matrices.

In [ ]:
fore_test_op_dense = joblib.load(open('/home/mitarb/fracarolli/eicu/final/sepsis_label/fore_test_op_susinf.pkl', 'rb'))
fore_test_ip_dense = joblib.load(open('/home/mitarb/fracarolli/eicu/final/sepsis_label/fore_test_ip_susinf.pkl', 'rb'))
test_input = fore_test_op_dense
test_ip = fore_test_ip_dense

mean_std_dict = joblib.load(open('/home/mitarb/fracarolli/eicu/final/mean_std_dict.pkl', 'rb'))
var_to_ind = dict()
with open("/home/mitarb/fracarolli/eicu/final/var_to_ind.csv") as file:
    for line in file:
        key, value = line.strip().split(",")
        var_to_ind[key]=int(value)

In [ ]:
print(test_input.shape)
print(len(test_ip[0]))
#fore_test_op_triplet = joblib.load(open('/home/mitarb/fracarolli/eicu/final/evaluation_scripts+pkl/fore_test_op_dense.pkl', 'rb'))
#fore_test_ip_triplet = joblib.load(open('fore_test_ip_dense.pkl', 'rb'))

In [ ]:
def get_sofa(matrix, var_to_ind): #24xV matrix
    # GCS: min_eye, min_motor, min_verbal = 5, 5, 5
    print(matrix.size())
    #raise Exception
    key ="GCS eye"
    var_to_ind = {x:i-1 for x,i in var_to_ind.items()}
    a=matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]
    print(a)
    min_eye = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=4)
    key = "GCS motor"
    min_motor = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=6)
    key = "GCS verbal"
    min_verbal = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=5)
    

    GCS = min_eye + min_motor + min_verbal
    if GCS > 14: GCS_sofa = 0
    elif GCS > 12: GCS_sofa = 1
    elif GCS > 9:  GCS_sofa = 2
    elif GCS > 5:  GCS_sofa = 3
    else: GCS_sofa = 4
    #print('GCS_sofa is', GCS_sofa, ';     GCS is', GCS,'; GCS eye', min_eye, '; GCS motor', min_motor, '; GCS verbal', min_verbal)

    key = "Bilirubin (Total)"
    bilir = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    if bilir > 12: bilir_sofa = 4
    elif bilir > 6: bilir_sofa = 3
    elif bilir > 2: bilir_sofa = 2
    elif bilir > 1.2: bilir_sofa = 1
    else: bilir_sofa = 0
    #print('bilir_Sofa is', bilir_sofa, ';   bilirubin is', bilir)
    
    # Coagulation (Platelets)
    key = "Platelets"
    plate = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=160)
    if plate > 150: plate_sofa = 0
    elif plate > 100: plate_sofa = 1
    elif plate > 50: plate_sofa = 2
    elif plate > 20: plate_sofa = 3
    else: plate_sofa = 4
    #print('plate_sofa is', plate_sofa, ';   platelet count is', plate)
    
    # print('Urinmenge 24h', sum(data_var[data_var['variable']=='Urine']['value2']))

    key = "Urine"
    urine = sum(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key = "Creatinine (Blood)"
    creat = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    
    if (urine < 200) or (creat > 5): renal_sofa = 4
    elif  (urine < 500) or (creat > 3.5): renal_sofa = 3
    elif creat > 2.0: renal_sofa = 2
    elif creat > 1.2: renal_sofa = 1
    else: renal_sofa = 0
    #print('renal_sofa:',renal_sofa,';       urine 24:',urine,'; creat:', creat)
    
    CS_data = get_CS(matrix, var_to_ind)
    cs_sofa = CS_SOFA(CS_data)
    
    #cs_sofa = 0
    key="FiO2"
    fio2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key="PaO2"
    po2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    PaO2FiO2 = 100*po2/fio2
    print("size", PaO2FiO2.size())
    PaO2FiO2 = PaO2FiO2[torch.nonzero(PaO2FiO2, as_tuple=True)]
    pao2fio2 = min(PaO2FiO2)
    if pao2fio2<100: resp=4
    elif pao2fio2<200: resp=3
    elif pao2fio2<300:resp=2
    elif pao2fio2<400:resp=1
    else: resp=0
    return GCS_sofa, cs_sofa, resp, plate_sofa, bilir_sofa, renal_sofa

def get_CS(matrix, var_to_ind):
    #data_var = data_pat[data_pat['variable'].isin(['Dobutamine','Dopamine','Epinephrine','Norepinephrine','Weight'])]
    #data_var['value2'] = data_var['value']*data_var['std']+data_var['mean']
    
    #weight = min(data_var[data_var['variable']=='Weight']['value2'], default=80)  # set default weight to 80kg.
    key = "Weight"
    weight = 80# min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)], default=80)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    key = "d Dopamine ratio"
    try:
        data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dop = data_dop /60/weight*1000
    except:
        data_dop = 0

    key = "d Dobutamine ratio"
    
    try:
        data_dobu = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dobu = data_dobu  /60/weight*1000
    except:
        data_dobu = 0
    key = "d Epinephrine ratio"
    
    try:
        data_epi = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_epi = data_epi  /60/weight*1000
    except:
        data_epi = 0
    key = "d Norepinephrine ratio"
    
    try:
        data_nore = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_nore = data_nore /60/weight*1000
    except:
        data_nore = 0
        


    key = "SBP"
    SBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    key = "DBP"
    DBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    MAP = 2/3 * DBP + 1/3 * SBP
    MAP = min(MAP[torch.nonzero(MAP, as_tuple=True)], default=100)
                 
        
    return MAP, data_dop, data_dobu, data_epi, data_nore 
    
def CS_SOFA(data):
    map = data[0]
    dop, dobu, epi, nore = data[1:5]
    # print('CS data: mdden', data)
    if (dop > 15) or (epi > 0.1) or (nore > 0.01): CS = 4
    elif (dop > 5) or (epi > 0) or (nore > 0): CS = 3
    elif (dop > 0) or (dobu > 0): CS = 2
    elif map < 70: CS = 1
    else: CS = 0
    # print('CS Sofa is:', CS)
    return CS 
    
factor_keys =["GCS eye", "GCS motor", "GCS verbal", "Bilirubin (Total)", "Platelets", "Urine", "Creatinine (Blood)", "FiO2", "PaO2", "d Dopamine ratio", "d Dobutamine ratio",
"d Epinephrine ratio", "d Norepinephrine ratio", "SBP", "DBP"]
factor_indices = sorted([var_to_ind[x]-1 for x in factor_keys])

In [ ]:
d_model, e_layers, d_layers, d_ff = 256, 2, 2, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')



#Michi: die ganzen Argparses könnte man noch anders implementieren, wollte es nur schnell zum laufen bringen.
import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')
args = parser.parse_args(args=[])
import importlib


import matplotlib.pyplot as plt

In [ ]:
import models.InformerAutoregressiveWithForcedDecoding as autoformer
importlib.reload(autoformer)
model = autoformer.Model(args).cuda()


# Crossinteractions

In [ ]:
"""
torch.cuda.empty_cache()
import models.InformerAutoregressiveWithForcedDecoding as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()


# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("mimic_iii_24h_strats_no_interp_with_ss_fore_informer_ims_fr_detach.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0

keys = ["Dopamine", "Dobutamine", "Norepinephrine"]

#data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

min_dict = dict()
max_dict = dict()
twentyfive_dict = dict()
median_dict = dict()
seventyfive_dict = dict()
matrix = torch.tensor(test_input)
for x in range(0, len(test_input)):

    for key in keys:
        try:
            min_data = min(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)])#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
            if key not in min_dict:
                min_dict[key] = min_data
            else:
                min_dict[key] = min([min_data, min_dict[key]])
        except:
            pass
        try:
            max_data = max(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)])#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in max_dict:
                max_dict[key] = max_data
            else:
                max_dict[key] = max([max_data, max_dict[key]])
        except:
            pass
        try:
            twentyfive_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.25)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in twentyfive_dict:
                twentyfive_dict[key] = twentyfive_data.item()
            else:
                twentyfive_dict[key] = max([twentyfive_data, twentyfive_dict[key]])
        except:
            pass
        try:
            median_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.50)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in median_dict:
                median_dict[key] = median_data.item()
            else:
                median_dict[key] = max([median_data, median_dict[key]])
        except:
            pass
        try:
            seventyfive_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.75)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in seventyfive_dict:
                seventyfive_dict[key] = seventyfive_data.item()
            else:
                seventyfive_dict[key] = max([seventyfive_data, seventyfive_dict[key]])
        except:
            pass
print(min_dict, max_dict)
resultdict = dict()
for key in keys:
    maxes, mins, twentyfives, medians, seventyfives = [], [], [], [], []
    for start in pbar:
        print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        #print(var_to_ind[key])
        #print(min_dict[key])
        #print(max_dict[key])
        with torch.no_grad():
            output_min = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: min_dict[key]})
            output_max = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: max_dict[key]})
            
            output_25 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: twentyfive_dict[key]})
            output_50 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: median_dict[key]})
            output_75 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: seventyfive_dict[key]})
        #print(output1, output2, output1==output2)
        for x in output_min:
            mins.append(x.cpu())
        for x in output_max:
            maxes.append(x.cpu())
        for x in output_25:
            twentyfives.append(x.cpu())
        for x in output_50:
            medians.append(x.cpu())
        for x in output_75:
            seventyfives.append(x.cpu())
    print(len(maxes), maxes[0].size())
    resultdict[(key, "min")]=mins
    resultdict[(key, "max")]=maxes
    resultdict[(key, "25")]=twentyfives
    resultdict[(key, "50")]=medians
    resultdict[(key, "75")]=seventyfives
import pickle

with open('crossinteractions.pickle', 'wb') as handle:
    pickle.dump([resultdict, var_to_ind, min_dict, max_dict, mean_std_dict], handle)
"""

# 8 Sofa Analysis IMS Informer TF

In [ ]:

d_model, e_layers, d_layers, d_ff = 256, 2, 2, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')


import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib

In [ ]:
import matplotlib.pyplot as plt

import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_mod08_Dense_IMS_TF.pytorch"))
model.eval()
y_true, y_pred = [], []
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()

    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)

se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))


model_name = '8&Dense&IMS&TF&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# 7 Sofa Analysis DMS Informer

In [ ]:

import models.Informer as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/final/240408_models/eicu_informer.pytorch"))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()

    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)

model_name = '7&Dense&DMS&-&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# 10 Sofa Analysis IMS Informer Backprop

In [ ]:

import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_mod10_Dense_IMS_SF_BP.pytorch"))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Informer_IMS_SF_B',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))

model_name = '10&Dense&IMS&SF&Yes&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# 9 Sofa Analysis IMS Informer Detach

In [ ]:

import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_mod09_Dense_IMS_SF_D.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
test_loss_sofa_variables = 0
sofa_losses = []
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Informer_IMS_SF_D',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))



model_name = '9&Dense&IMS&SF&No&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# Autof SOFA Analysis Autoformer

In [ ]:

#Michi: die ganzen Argparses könnte man noch anders implementieren, wollte es nur schnell zum laufen bringen.
import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 

parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')



args = parser.parse_args(args=[])
import importlib


import models.Autoformer as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_autoformer.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])

    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix[:, :, :V], input_mark, dec_inp, output_mark)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Autoformer_DMS',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))



model_name = 'Autoformer&Dense&DMS&-&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# DLinear SOFA Analysis DLINEAR

In [ ]:

#Michi: die ganzen Argparses könnte man noch anders implementieren, wollte es nur schnell zum laufen bringen.
import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=262, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib



import models.DLinear as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_dlinear.pytorch"))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24, :V]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('DLinear_DMS',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))


model_name = 'DLinear&Dense&DMS&-&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# Linear SOFA Analysis Linear

In [ ]:

import models.Linear as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_linear.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24, :V]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Linear_DMS',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))


model_name = 'Linear&Dense&DMS&-&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# 1 STRATS DMS

In [ ]:
sample_divisor = 1
V=98
fore_max_len=880*3
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=False):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, D * d),
                                      nn.Tanh(),
                                      nn.Linear(D * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, V)
            self.dec = nn.TransformerDecoderLayer(d_model=V, dim_feedforward=V, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, V).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(linear_memory.size(0), 1, V).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, V).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(17, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()

# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model
best_val_loss = np.inf

fore_savepath = '/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_mod01_Triplet_DMS.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")
model.load_state_dict(torch.load(fore_savepath))
model.eval()
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
counterblub = 0
for start in tqdm(range(0, len(test_ip[0]), 32)):
    with torch.no_grad():
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(test_ip)], trainn=False)#, tgt=torch.tensor(test_ip[start:start+batch_size], dtype=torch.float32).cuda())
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24, :V]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(batch_size, 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    print(output_mask.size(), output.size(), output_matrix.size())
    loss = output_mask[:, -24:, :]*(
        output-output_matrix[:, -24:, :])**2
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            counterblub+=1
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Triplet_DMS',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))

model_name = '1&Triplet&DMS&-&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'

print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='') 
print(counterblub)

# 2 STRATS IMS Teacher Forcing

In [ ]:

class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, D * d),
                                      nn.Tanh(),
                                      nn.Linear(D * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, V)
            self.dec = nn.TransformerDecoderLayer(d_model=V, dim_feedforward=V, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, V).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(linear_memory.size(0), 1, V).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(cont_emb.size(0), 1, V).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x


lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(17, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()

# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf

fore_savepath = '/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_mod02_Triplet_IMS_TF.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")
model.load_state_dict(torch.load(fore_savepath))
model.eval()
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in tqdm(range(0, len(test_ip[0]), 32)):
    #print(start, start+batch_size)
    with torch.no_grad():
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(test_ip)], trainn=False)#, tgt=torch.tensor(test_ip[start:start+batch_size], dtype=torch.float32).cuda())
    
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(batch_size, 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
                
            TP => TP
            FP => FN
            FN => TN
            TN => FP
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Triplet_IMS_TF',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))

model_name = '2&Triplet&IMS&TF&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# 4 STRATS IMS Backprop

In [ ]:

class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, D * d),
                                      nn.Tanh(),
                                      nn.Linear(D * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, V)
            self.dec = nn.TransformerDecoderLayer(d_model=V, dim_feedforward=V, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, V).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(cont_emb.size(0), 1, V).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(cont_emb.size(0), 1, V).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x


lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(17, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()

# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf

fore_savepath = '/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_mod04_Triplet_IMS_SF_BP.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")
model.load_state_dict(torch.load(fore_savepath))
model.eval()
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in tqdm(range(0, len(test_ip[0]), 32)):
    with torch.no_grad():
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(test_ip)], trainn=False)#, tgt=torch.tensor(test_ip[start:start+batch_size], dtype=torch.float32).cuda())
    
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(batch_size, 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Triplet_IMS_SF_B',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))


model_name = '4&Triplet&IMS&SF&Yes&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# 3 STRATS IMS Detach

In [ ]:

class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, D * d),
                                      nn.Tanh(),
                                      nn.Linear(D * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, V)
            self.dec = nn.TransformerDecoderLayer(d_model=V, dim_feedforward=V, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, V).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(cont_emb.size(0), 1, V).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(cont_emb.size(0), 1, V).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(17, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()


optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model
best_val_loss = np.inf

fore_savepath = '/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_24h_strats_no_interp_with_ss_fore_strats_ims_sampling_detach.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")
model.load_state_dict(torch.load(fore_savepath))
model.eval()
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in tqdm(range(0, len(test_ip[0]), 32)):
    #print(start, start+batch_size)
    with torch.no_grad():
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(test_ip)], trainn=False)#, tgt=torch.tensor(test_ip[start:start+batch_size], dtype=torch.float32).cuda())
    
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(batch_size, 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Triplet_IMS_SF_D',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))

model_name = '3&Triplet&IMS&SF&No&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# Informer with Attention-DMS

In [ ]:

#Michi: die ganzen Argparses könnte man noch anders implementieren, wollte es nur schnell zum laufen bringen.
import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib


import matplotlib.pyplot as plt

In [ ]:

import models.InformerFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/240401_Experimente/eicu_mod07_Dense_DMS_cont.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()

    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a=(get_sofa(outpu*mask, var_to_ind))
        b=(get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)

print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p)

model_name = 'Dense&Dense&DMS&-&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

In [ ]:
import os
import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
for model_path in os.listdir():
    if "eicu_mod10_Dense" not in model_path: continue

    model.load_state_dict(torch.load(model_path))
    model.eval()
    y_true, y_pred = [], []
    print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    sofa_losses = []
    test_loss_sofa_variables = 0
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
            loss = output_mask[:, -args.pred_len:, :]*(
            output-output_matrix[:, -args.pred_len:, :])**2
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
            a = (get_sofa(outpu*mask, var_to_ind))
            b = (get_sofa(real, var_to_ind))
            c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
            prediction_sofa_difference = (sum(a)-sum(c)) >= 2
            gold_sofa_difference = (sum(b)-sum(c)) >= 2
            accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
            if gold_sofa_difference:
                if prediction_sofa_difference: TP +=1
                else: FP += 1
            else:
                if prediction_sofa_difference: TN +=1
                else: FN += 1
            c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
            sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
        loss_old = loss
        loss = loss.sum(axis=-1).mean(axis=-1).sum()
        loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
        loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
        test_loss+=loss
        test_loss_part1+=loss_part1
        test_loss_part2+=loss_part2
        test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
        test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
        test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
        test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
        test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
    test_loss_p = test_loss/len(test_input)
    se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
    test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
    test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
    test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
    sofa_loss = sum(sofa_losses)/len(sofa_losses)
    accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
    se_accuracy_sepsis = np.sqrt(accuracy_sepsi/tes * (1-accuracy_sepsis)/len(test_input))
    se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
    se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
    se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
    print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
    print('Informer_IMS_SF_B',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))

    model_name = model_path
    MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
    SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
    MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
    MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
    SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
    print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
    print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
    print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
    print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

# NEW EXPERIMENTS

In [ ]:
d_model, e_layers, d_layers, d_ff = 256, 2, 2, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')



import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib

In [ ]:
import os
import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2

for model_path in os.listdir("/home/mitarb/fracarolli/eicu/final/240408_models"):
    if not("mod10" in model_path or "mod09" in model_path): continue
    print(model_path)
    model = autoformer.Model(args).cuda()
    # Load pretrained weights here.
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    #fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
    scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
    model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/final/240408_models/" + model_path))
    #model.load_state_dict(torch.load(fore_savepath))
    model.eval()
    y_true, y_pred = [], []
    print(test_input)
    pbar = tqdm(range(0, len(test_input), batch_size))
    loss_list = []
    test_loss = 0
    test_loss_part1 = 0
    test_loss_part2 = 0
    test_loss_sofa_variables = 0
    sofa_losses = []
    accuracy_list = []
    TP, TN, FP, FN = 0, 0, 0, 0
    test_loss_sum_square_total = 0
    test_loss_part1_sum_square_total = 0
    test_loss_part2_sum_square_total = 0
    test_loss_sofa_sum_square_total = 0
    for start in pbar:
        print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, V:]

        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
        #torch.Size([32, 24, 129])
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
            loss = output_mask[:, -args.pred_len:, :]*(
            output-output_matrix[:, -args.pred_len:, :])**2
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
            a = (get_sofa(outpu*mask, var_to_ind))
            b = (get_sofa(real, var_to_ind))
            c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
            prediction_sofa_difference = (sum(a)-sum(c)) >= 2
            gold_sofa_difference = (sum(b)-sum(c)) >= 2
            accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
            if gold_sofa_difference:
                if prediction_sofa_difference: TP +=1
                else: FP += 1
            else:
                if prediction_sofa_difference: TN +=1
                else: FN += 1
            c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
            sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
        loss_old = loss
        loss = loss.sum(axis=-1).mean(axis=-1).sum()
        loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
        loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
        test_loss+=loss
        test_loss_part1+=loss_part1
        test_loss_part2+=loss_part2
        test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
        test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
        test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
        test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
        test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
    test_loss_p = test_loss/len(test_input)
    se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
    test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
    test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
    test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
    sofa_loss = sum(sofa_losses)/len(sofa_losses)
    accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
    se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
    se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
    se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
    se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
    print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
    print('Informer_IMS_SF_D',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))



    model_name = model_path+'&'
    MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
    SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
    MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
    MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
    SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
    print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
    print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
    print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
    print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

In [ ]:
d_model, e_layers, d_layers, d_ff = 256, 2, 2, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')



import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')


# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 

parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib

In [ ]:
import os
import models.InformerDMSFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/final/240408_models/eicu_informer.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()

    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a=(get_sofa(outpu*mask, var_to_ind))
        b=(get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)

print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p)

model_name = 'Informer&Dense&DMS&-&-&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

In [ ]:
d_model, e_layers, d_layers, d_ff = 512, 2, 1, 2048
number_of_epochs = 100
V, D = 98, 17
fore_max_len = 2640
print('loading of joblib completed')



import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size')
parser.add_argument('--d_model', type=int, default=d_model, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=e_layers, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=d_layers, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=d_ff, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib

In [ ]:

import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/final/240408_models/eicu_mod_dense_SS_RI_D.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
test_loss_sofa_variables = 0
sofa_losses = []
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Informer_IMS_SF_D',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))



model_name = '9&Dense&IMS&SS-RI&No&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

In [ ]:

import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("/home/mitarb/fracarolli/eicu/final/240408_models/eicu_mod_dense_SS_RI_BP.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
test_loss_sofa_variables = 0
sofa_losses = []
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, V:]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Informer_IMS_SF_D',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))



model_name = '9&Dense&IMS&SS-RI&Yes&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

In [ ]:
sample_divisor=1
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, D * d),
                                      nn.Tanh(),
                                      nn.Linear(D * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, V)
            self.dec = nn.TransformerDecoderLayer(d_model=V, dim_feedforward=V, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, V).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(cont_emb.size(0), 1, V).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(cont_emb.size(0), 1, V).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(17, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()

# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf

fore_savepath = '/home/mitarb/fracarolli/eicu/final/240408_models/eicu_mod_triplet_SS_RI_BP.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")
model.load_state_dict(torch.load(fore_savepath))
model.eval()
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in tqdm(range(0, len(test_ip[0]), 32)):
    with torch.no_grad():
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(test_ip)], trainn=False)#, tgt=torch.tensor(test_ip[start:start+batch_size], dtype=torch.float32).cuda())
    
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(batch_size, 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Triplet_IMS_SF_B',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))


model_name = '4&Triplet&IMS&SS-RI&Yes&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))

In [ ]:

class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, D * d),
                                      nn.Tanh(),
                                      nn.Linear(D * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, V)
            self.dec = nn.TransformerDecoderLayer(d_model=V, dim_feedforward=V, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, V).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(cont_emb.size(0), 1, V).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(cont_emb.size(0), 1, V).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(17, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()

# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf

fore_savepath = '/home/mitarb/fracarolli/eicu/final/240408_models/eicu_mod_triplet_SS_RI_D.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")
model.load_state_dict(torch.load(fore_savepath))
model.eval()
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
sofa_losses = []
test_loss_sofa_variables = 0
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in tqdm(range(0, len(test_ip[0]), 32)):
    with torch.no_grad():
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(test_ip)], trainn=False)#, tgt=torch.tensor(test_ip[start:start+batch_size], dtype=torch.float32).cuda())
    
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]
    output_matrix = matrix[:, 24:, :V]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(batch_size, 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, V:]
    #torch.Size([32, 24, 129])
    loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Triplet_IMS_SF_B',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))


model_name = '4&Triplet&IMS&SS-RI&No&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(model_name,MSE_string, SOFA_string, sep='',file=open('eicu_table_sample_sepsis1.txt','a'))
print(model_name,MSE8_string, MSE16_string, sep='',file=open('eicu_table_sample_sepsis2.txt','a'))
print(model_name,TP,'&',TN,'&',FP,'&',FN,'&', TP+TN+FP+FN,'&', len(test_input), sep='',file=open('eicu_table_sample_sepsis3.txt','a')) 
print(model_name,len(test_input),'&',SOFA_acc_string, sep='',file=open('eicu_table_sample_sepsis4.txt','a'))